<a href="https://colab.research.google.com/github/ahmadluay9/ADK-Training/blob/main/adk_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

# 1. Setup

In [ ]:
# !pip install google-adk google-genai

## 1.1 GCP Configuration

In [ ]:
# Set your project ID
project_id = 'your-project-name' # @param {type:"string"}
location = 'us-central1' # @param {type:"string"}

!gcloud config set project {project_id}

In [ ]:
project_id_output = !gcloud config get project
print(f"PROJECT_ID = {project_id_output[0]}")
print("")
!gcloud auth list

## 1.2 ADK Configurations

In [ ]:
import os

try:
  os.environ['GOOGLE_CLOUD_PROJECT'] = project_id # The SDK uses this ID to route requests to the correct project for usage tracking and billing.
  os.environ['GOOGLE_CLOUD_LOCATION'] = location # Defines the specific geographic region where the Vertex AI resources (models) are hosted (e.g., us-central1, asia-southeast1).
  os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = "1" # Directs the SDK to use Vertex AI instead of the public Gemini API
except Exception as e:
    print(
        f"🔑 Authentication Error. Details: {e}"
    )

#

## 1.3 Import ADK components
Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

## 1.4: Helper functions

In [ ]:
import subprocess
from google.colab import output
from IPython.display import display, HTML

# Gets the proxied URL in the Google Colab environment
def get_adk_proxy_url(port=8000):
    """
    Starts the ADK web server in the background and generates a proxy URL
    to open it in Google Colab.
    """
    try:
        # Colab specific method to get the public URL for a local port
        url = output.eval_js(f'google.colab.kernel.proxyPort({port})')
    except Exception as e:
        print(f"Error determining proxy URL: {e}")
        return None

    # Automatically start the server in the background
    # We use shell=True to ensure the 'adk' command is found in the environment
    # and redirect output to a file so it doesn't clutter the notebook.
    adk_command = f'adk web --port {port} --allow_origins "*"'

    # Run the command asynchronously
    subprocess.Popen(adk_command, shell=True)

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #28a745; border-radius: 8px; background-color: #f0fff4; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #155724; font-size: 1.1em;">
            <strong>✅ Server Started Automatically</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #155724; line-height: 1.5;">
            The ADK web UI is running in the background. You don't need to run another cell.
            <ul style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;">Click the button below to open the UI.</li>
            </ul>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url

print("✅ Colab helper functions defined.")

## 1.5: Configure Retry Options
When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

---

# 🤖 Section 2: Your first AI Agent with ADK

## 🤔 2.1 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

## 2.2 Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [ ]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

## 2.3 Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [ ]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as adk run, adk web, or adk api_server. To learn more, check out the documentation related to runtime in ADK.

**b. Now you can call the .run_debug() method to send our prompt and get an answer.**

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [ ]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)

You can see a summary of ADK and its available languages in the response.

### 2.4 How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

### 🚀 2.5 Your Turn!

This is your chance to see the agent in action. Ask it a question that requires current information.

Try one of these, or make up your own:

- What's the weather in London?
- Who won the last soccer world cup?
- What new movies are showing in theaters now?

In [ ]:
response = await runner.run_debug("What's the weather in Jakarta?")

---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to create a folder named `my-first-agent`. This folder will include:

- `agent.py` – the main file where you implement your agent logic

- `.env` – preconfigured with:

  - `GOOGLE_CLOUD_PROJECT`

  - `GOOGLE_CLOUD_LOCATION`

  - `GOOGLE_GENAI_USE_VERTEXAI`

- `__init__.py` – initializes the agent module

This setup provides everything you need to start building and testing your agent through the web interface.

In [ ]:
!adk create --help

In [ ]:
# TODO: Change --project value to your real GCP project ID (e.g., my-gcp-project-123)
!adk create my_first_agent --model gemini-2.5-flash-lite --project your-project-name --region us-central1

Now we can run ADK web:

In [ ]:
url_prefix = get_adk_proxy_url()

Now you can access the ADK dev UI using the link above.

Once you open the link, you'll see the ADK web interface where you can ask your ADK agent questions.

**Note**: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.